In [11]:
import os
import json
import re

In [10]:
text_data = [" ".join([i["text"] for topic in j["topics"] for i in topic["items"]]) for j in json_data]
len(text_data)

215

In [ ]:
cleaning = (re.sub("[^A-Za-z']+", ' ', str(row)).lower() for row in df['spoken_words'])